# Deutsch-Jozsa Algorithm: Exponential Quantum Speedup

## Introduction

The Deutsch-Jozsa algorithm, developed by David Deutsch and Richard Jozsa in 1992, generalizes Deutsch's algorithm to $n$ qubits. While Deutsch's algorithm showed a factor-of-2 speedup, Deutsch-Jozsa demonstrates an **exponential speedup** over the best deterministic classical algorithm.

This tutorial builds on the concepts from the Deutsch algorithm notebook. If you haven't reviewed it yet, we recommend starting there to understand the fundamental principles of quantum parallelism and phase kickback.

## The Problem

Given a black-box function (oracle) $f: \{0,1\}^n \rightarrow \{0,1\}$ that takes $n$ bits as input and produces 1 bit as output, we are **promised** that $f$ is either:

- **Constant**: $f(x) = c$ for all $x$ (where $c \in \{0, 1\}$)
- **Balanced**: $f(x) = 0$ for exactly half of the inputs and $f(x) = 1$ for the other half

**Goal**: Determine whether $f$ is constant or balanced.

### Classical Complexity

In the worst case, a deterministic classical algorithm must evaluate:
$$\text{Queries} = 2^{n-1} + 1$$

Why? Consider $n = 3$ (8 possible inputs):
- If the function is balanced, it outputs 0 for 4 inputs and 1 for 4 inputs
- In the worst case, you might query $2^{n-1} = 4$ inputs and get the same answer
- You need one more query to confirm it's constant (all same) or balanced (finally different)

For $n = 10$: need up to **513 queries**  
For $n = 20$: need up to **524,289 queries**

### Quantum Complexity

The Deutsch-Jozsa algorithm solves this with **exactly 1 query** to the oracle, regardless of $n$!

This is an **exponential speedup**: $O(2^n)$ classical vs $O(1)$ quantum.

## Review: Deutsch's Algorithm (n=1)

Deutsch's algorithm is the special case where $n=1$:
- Input: 1 bit
- 4 possible functions (2 constant, 2 balanced)
- Classical: 2 queries needed
- Quantum: 1 query needed

Deutsch-Jozsa extends this to $n$ bits:
- Input: $n$ bits
- $2^{2^n}$ possible functions (only 2 constant, many balanced)
- Classical: up to $2^{n-1} + 1$ queries
- Quantum: 1 query

## The Quantum Circuit

The circuit structure is identical to Deutsch's algorithm, but with $n$ input qubits:

```
|0⟩ ─── H ─── ┐
|0⟩ ─── H ─── │
|0⟩ ─── H ─── ┤  Uf  ├─── H ─── H ─── H ─── Measure
  ⋮      ⋮    │      │    ⋮     ⋮     ⋮
|0⟩ ─── H ─── ┤      ├─── H ─── H ─── H ─── Measure
|1⟩ ─── H ─── ┘      └
```

### Algorithm Steps:

1. **Initialize**: $n$ qubits to $|0\rangle$ and 1 ancilla qubit to $|1\rangle$
2. **Hadamard all qubits**: Create superposition over all $2^n$ inputs
3. **Apply oracle $U_f$**: Query all inputs simultaneously
4. **Hadamard the input qubits**: Interference step
5. **Measure input qubits**:
   - All measure $|0\rangle$ → **Constant**
   - Any measure $|1\rangle$ → **Balanced**

### Why It Works

After the Hadamards and oracle, the state becomes:
$$|\psi\rangle = \frac{1}{2^n} \sum_{x=0}^{2^n-1} (-1)^{f(x)} |x\rangle$$

The final Hadamards perform an interference measurement:
- **Constant**: All amplitudes interfere constructively at $|0\cdots0\rangle$
- **Balanced**: Amplitudes cancel at $|0\cdots0\rangle$ due to equal positive/negative phases

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import matplotlib.pyplot as plt
import numpy as np

print("Qiskit imported successfully!")

## Oracle Implementations

Let's implement oracles for various $n$-qubit constant and balanced functions.

In [ ]:
def constant_oracle(n, output=0):
    """
    Creates a constant oracle: f(x) = output for all x
    
    Args:
        n: Number of input qubits
        output: Constant output value (0 or 1)
    
    Returns:
        QuantumCircuit: Oracle circuit
    """
    oracle = QuantumCircuit(n + 1, name=f'f(x)={output}')
    
    if output == 1:
        # Flip the output qubit to make f(x) = 1 for all x
        oracle.x(n)
    # If output == 0, do nothing (identity)
    
    return oracle


def balanced_oracle(n, pattern='alternating'):
    """
    Creates a balanced oracle where f(x) = 0 for half the inputs
    and f(x) = 1 for the other half.
    
    Args:
        n: Number of input qubits
        pattern: Type of balanced function
            - 'alternating': f(x) = x_0 (depends on LSB)
            - 'parity': f(x) = x_0 ⊕ x_1 ⊕ ... ⊕ x_{n-1}
            - 'random': random balanced function
    
    Returns:
        QuantumCircuit: Oracle circuit
    """
    oracle = QuantumCircuit(n + 1, name=f'balanced-{pattern}')
    
    if pattern == 'alternating':
        # f(x) depends on the least significant bit (qubit 0)
        oracle.cx(0, n)
        
    elif pattern == 'parity':
        # f(x) = XOR of all input bits
        for i in range(n):
            oracle.cx(i, n)
            
    elif pattern == 'random':
        # Create a random balanced function
        # This is done by applying CNOTs from random qubits
        # and inserting X gates to ensure it's balanced
        np.random.seed(42)  # For reproducibility
        
        # Apply CNOT from a random subset of qubits
        for i in range(n):
            if np.random.random() > 0.5:
                oracle.cx(i, n)
        
        # Ensure at least one CNOT for balanced property
        if oracle.num_parameters == 0:  # If no CNOTs were added
            oracle.cx(0, n)
    
    return oracle

## Building the Deutsch-Jozsa Circuit

In [ ]:
def deutsch_jozsa(oracle, n):
    """
    Implements the Deutsch-Jozsa algorithm.
    
    Args:
        oracle: Quantum circuit implementing f(x)
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Complete Deutsch-Jozsa circuit
    """
    # Create circuit with n input qubits, 1 output qubit, n classical bits
    qr = QuantumRegister(n + 1, 'q')
    cr = ClassicalRegister(n, 'c')
    dj_circuit = QuantumCircuit(qr, cr)
    
    # Step 1: Initialize output qubit to |1⟩
    dj_circuit.x(n)
    dj_circuit.barrier()
    
    # Step 2: Apply Hadamard gates to all qubits
    for i in range(n + 1):
        dj_circuit.h(i)
    dj_circuit.barrier()
    
    # Step 3: Apply the oracle
    dj_circuit.compose(oracle, inplace=True)
    dj_circuit.barrier()
    
    # Step 4: Apply Hadamard gates to input qubits
    for i in range(n):
        dj_circuit.h(i)
    dj_circuit.barrier()
    
    # Step 5: Measure input qubits
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

## Example 1: Small Case (n=2)

Let's start with $n=2$ input qubits (4 possible inputs).

In [ ]:
n = 2
print(f"Testing Deutsch-Jozsa with n={n} qubits")
print(f"Classical worst case: {2**(n-1) + 1} queries")
print(f"Quantum: 1 query\n")
print("=" * 60)

# Test different oracles
test_oracles = [
    ("Constant f(x)=0", constant_oracle(n, 0)),
    ("Constant f(x)=1", constant_oracle(n, 1)),
    ("Balanced (alternating)", balanced_oracle(n, 'alternating')),
    ("Balanced (parity)", balanced_oracle(n, 'parity')),
]

simulator = AerSimulator()
results_n2 = {}

for name, oracle in test_oracles:
    circuit = deutsch_jozsa(oracle, n)
    job = simulator.run(circuit, shots=1000)
    result = job.result()
    counts = result.get_counts()
    results_n2[name] = counts
    
    # Check if all measurements are 0
    all_zero = all(k == '0' * n for k in counts.keys())
    function_type = "CONSTANT" if all_zero else "BALANCED"
    
    print(f"{name}:")
    print(f"  Result: {function_type}")
    print(f"  Measurements: {counts}")
    print("-" * 60)

In [ ]:
# Visualize the results for n=2
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (name, counts) in enumerate(results_n2.items()):
    plot_histogram(counts, ax=axes[idx])
    axes[idx].set_title(f'{name}')

plt.tight_layout()
plt.show()

## Example 2: Medium Case (n=4)

With $n=4$, there are 16 possible inputs. Classically, we'd need up to 9 queries in the worst case.

In [ ]:
n = 4
print(f"Testing Deutsch-Jozsa with n={n} qubits")
print(f"Total possible inputs: {2**n}")
print(f"Classical worst case: {2**(n-1) + 1} queries")
print(f"Quantum: 1 query\n")
print("=" * 60)

test_oracles_n4 = [
    ("Constant f(x)=0", constant_oracle(n, 0)),
    ("Constant f(x)=1", constant_oracle(n, 1)),
    ("Balanced (alternating)", balanced_oracle(n, 'alternating')),
    ("Balanced (parity)", balanced_oracle(n, 'parity')),
    ("Balanced (random)", balanced_oracle(n, 'random')),
]

results_n4 = {}

for name, oracle in test_oracles_n4:
    circuit = deutsch_jozsa(oracle, n)
    job = simulator.run(circuit, shots=1000)
    result = job.result()
    counts = result.get_counts()
    results_n4[name] = counts
    
    all_zero = all(k == '0' * n for k in counts.keys())
    function_type = "CONSTANT" if all_zero else "BALANCED"
    
    print(f"{name}:")
    print(f"  Result: {function_type}")
    print(f"  Measurement outcomes: {len(counts)} unique states")
    print("-" * 60)

In [ ]:
# Visualize circuits for n=4
print("Circuit for Constant Oracle (n=4):")
const_circuit = deutsch_jozsa(constant_oracle(4, 0), 4)
const_circuit.draw('mpl', fold=-1)

In [ ]:
print("Circuit for Balanced Oracle - Parity (n=4):")
balanced_circuit = deutsch_jozsa(balanced_oracle(4, 'parity'), 4)
balanced_circuit.draw('mpl', fold=-1)

## Demonstrating Exponential Speedup

Let's compare the classical vs quantum query complexity for different values of $n$.

In [ ]:
# Compare query complexity
n_values = range(1, 11)
classical_queries = [2**(n-1) + 1 for n in n_values]
quantum_queries = [1] * len(n_values)

plt.figure(figsize=(12, 6))
plt.semilogy(n_values, classical_queries, 'ro-', label='Classical (worst case)', linewidth=2, markersize=8)
plt.semilogy(n_values, quantum_queries, 'bs-', label='Quantum (Deutsch-Jozsa)', linewidth=2, markersize=8)
plt.xlabel('Number of input qubits (n)', fontsize=12)
plt.ylabel('Number of oracle queries (log scale)', fontsize=12)
plt.title('Deutsch-Jozsa Algorithm: Exponential Speedup', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xticks(n_values)

# Add annotations
for n, cq in zip([5, 10], [classical_queries[4], classical_queries[9]]):
    plt.annotate(f'n={n}: {cq} queries', 
                xy=(n, cq), xytext=(n+0.5, cq*2),
                arrowprops=dict(arrowstyle='->', color='red', alpha=0.7),
                fontsize=10, color='red')

plt.tight_layout()
plt.show()

print("\nQuery Complexity Comparison:")
print("=" * 50)
print(f"{'n':<5} {'Inputs':<12} {'Classical':<15} {'Quantum':<10} {'Speedup'}")
print("=" * 50)
for n in [1, 2, 3, 4, 5, 10, 15, 20]:
    inputs = 2**n
    classical = 2**(n-1) + 1
    quantum = 1
    speedup = classical / quantum
    print(f"{n:<5} {inputs:<12} {classical:<15} {quantum:<10} {speedup:.0f}x")

## Example 3: Large Case (n=8)

With $n=8$ qubits:
- Total inputs: 256
- Classical worst case: 129 queries
- Quantum: 1 query

This demonstrates a **129x speedup**!

In [ ]:
n = 8
print(f"Testing Deutsch-Jozsa with n={n} qubits")
print(f"Total possible inputs: {2**n}")
print(f"Classical worst case: {2**(n-1) + 1} queries")
print(f"Quantum: 1 query")
print(f"Speedup: {2**(n-1) + 1}x\n")
print("=" * 60)

# Test a few oracles
test_cases = [
    ("Constant f(x)=0", constant_oracle(n, 0)),
    ("Balanced (parity)", balanced_oracle(n, 'parity')),
]

for name, oracle in test_cases:
    circuit = deutsch_jozsa(oracle, n)
    
    print(f"\n{name}:")
    print(f"  Circuit depth: {circuit.depth()}")
    print(f"  Circuit gates: {circuit.size()}")
    
    job = simulator.run(circuit, shots=1000)
    result = job.result()
    counts = result.get_counts()
    
    all_zero = all(k == '0' * n for k in counts.keys())
    function_type = "CONSTANT" if all_zero else "BALANCED"
    
    print(f"  Result: {function_type}")
    print(f"  Success rate: {max(counts.values()) / 1000 * 100:.1f}%")
    print("-" * 60)

## Understanding the Mathematics

### Initial State
After initialization and Hadamards:
$$|\psi_1\rangle = \frac{1}{2^{n/2}} \sum_{x=0}^{2^n-1} |x\rangle \otimes \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$$

### After Oracle
The oracle applies: $|x\rangle|y\rangle \rightarrow |x\rangle|y \oplus f(x)\rangle$

Using phase kickback with $|-\rangle$:
$$|\psi_2\rangle = \frac{1}{2^{n/2}} \sum_{x=0}^{2^n-1} (-1)^{f(x)} |x\rangle \otimes |-\rangle$$

### After Final Hadamards
The Hadamard transform on $n$ qubits:
$$H^{\otimes n}|x\rangle = \frac{1}{2^{n/2}} \sum_{z=0}^{2^n-1} (-1)^{x \cdot z} |z\rangle$$

Applying this:
$$|\psi_3\rangle = \frac{1}{2^n} \sum_{x=0}^{2^n-1} \sum_{z=0}^{2^n-1} (-1)^{f(x) + x \cdot z} |z\rangle$$

### Probability of Measuring All Zeros
$$P(|0\cdots0\rangle) = \left|\frac{1}{2^n} \sum_{x=0}^{2^n-1} (-1)^{f(x)}\right|^2$$

- **Constant function**: All terms have the same sign → sum = $\pm 2^n$ → $P = 1$
- **Balanced function**: Half positive, half negative → sum = 0 → $P = 0$

## Key Insights

### 1. Quantum Parallelism
The $n$-qubit Hadamard transform creates a superposition over all $2^n$ possible inputs:
$$H^{\otimes n}|0\cdots0\rangle = \frac{1}{2^{n/2}} \sum_{x=0}^{2^n-1} |x\rangle$$

When we apply the oracle, it evaluates $f(x)$ for **all** $2^n$ inputs simultaneously!

### 2. Global vs Local Information
- Classical algorithms must query individual input-output pairs: $f(0), f(1), f(2), \ldots$
- Deutsch-Jozsa extracts a **global property** (constant vs balanced) without learning individual values
- This is achieved through quantum interference

### 3. Interference Pattern
- **Constant**: All $2^n$ contributions interfere constructively at $|0\cdots0\rangle$
- **Balanced**: Exactly half have positive phase, half negative → perfect cancellation at $|0\cdots0\rangle$

### 4. Deterministic Algorithm
Unlike some quantum algorithms that give probabilistic answers, Deutsch-Jozsa is **deterministic**:
- Always gives the correct answer
- With 100% probability
- In a single query

## Practical Limitations

While Deutsch-Jozsa demonstrates exponential speedup, it has some limitations:

1. **Promise Problem**: We're promised the function is either constant or balanced. In practice, most functions are neither.

2. **Oracle Model**: The algorithm assumes we have access to a quantum oracle. Building such an oracle might be expensive.

3. **Limited Practical Use**: The specific problem (constant vs balanced) isn't commonly encountered in real applications.

However, Deutsch-Jozsa is important because:
- It proves quantum computers can achieve exponential speedup
- It introduces techniques used in practical algorithms (e.g., Grover's, Shor's)
- It demonstrates the power of quantum interference and global property extraction

## Comparison with Other Algorithms

| Algorithm | Problem | Classical | Quantum | Speedup |
|-----------|---------|-----------|---------|----------|
| Deutsch | 1-bit function | 2 queries | 1 query | 2x |
| Deutsch-Jozsa | n-bit function | $2^{n-1}+1$ | 1 query | Exponential |
| Grover | Unstructured search | $O(N)$ | $O(\sqrt{N})$ | Quadratic |
| Shor | Factoring | $O(e^{n^{1/3}})$ | $O(n^3)$ | Exponential |

Deutsch-Jozsa was one of the first algorithms to demonstrate that quantum computers can solve certain problems exponentially faster than classical computers.

## Conclusion

The Deutsch-Jozsa algorithm elegantly demonstrates:

1. **Exponential Quantum Speedup**: From $O(2^n)$ classical queries to $O(1)$ quantum queries

2. **Quantum Parallelism**: Evaluating $2^n$ inputs simultaneously through superposition

3. **Quantum Interference**: Using constructive/destructive interference to extract global properties

4. **Phase Kickback**: Encoding function outputs as quantum phases

5. **Deterministic Quantum Algorithm**: 100% success rate, unlike probabilistic classical algorithms

This algorithm, while not practically useful for real-world problems, serves as a crucial proof-of-concept that quantum computers can outperform classical computers and laid the groundwork for more powerful quantum algorithms.

### Next Steps

To build on these concepts, explore:
- **Bernstein-Vazirani Algorithm**: Finds a hidden bit string with similar techniques
- **Simon's Algorithm**: Demonstrates exponential speedup for a different promise problem
- **Grover's Algorithm**: Practical quadratic speedup for database search
- **Shor's Algorithm**: Exponential speedup for integer factorization